In [1]:
!pip3 install pandas h3 sqlalchemy boto3 oci delta-spark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
print("starting...")



# -*- coding: utf-8 -*-
"""part_1_mobility_script_generic(new_dataset)_new (2).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1dGNHBaJAS1w2hsZw4c_Kx4bFdQ0Kzdg_
"""

print("starting...")

from pyspark.sql.window import Window 
import pyspark
from pyspark import SparkContext

from pyspark.sql import functions as F
import pyspark.sql.functions as func


from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import col, window
import pyspark.sql.functions as func
from pyspark.sql.functions import desc
import datetime, time
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as sf

import datetime as dt
import numpy as np
import os

from multiprocessing.pool import Pool
import pickle
import pandas as pd

from h3 import h3


from math import radians, cos, sin, asin, sqrt
from pyspark.sql.functions import lit
import json

from datetime import datetime, date, timedelta
 
import oci
from pyspark.sql import SparkSession
import sys
import boto3
from io import BytesIO

from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.functions import round, col
from dateutil import tz
import sqlalchemy as db

import pyspark

from pyspark.sql import SparkSession
ociconf = oci.config.from_file()

spark = SparkSession.builder \
        .config("spark.driver.memory", "4g") \
        .config("spark.driver.cores", "4") \
        .config("spark.executor.memory", "12g") \
        .config("spark.executor.cores", "4") \
        .config("spark.scheduler.mode", "FAIR") \
        .config("spark.cores.max", "16") \
        .getOrCreate()

# spark.conf.set('fs.oci.client.auth.tenantId', ociconf['tenancy'])
# spark.conf.set('fs.oci.client.auth.userId',  ociconf['user'])
# spark.conf.set('fs.oci.client.auth.fingerprint', ociconf['fingerprint'])
# spark.conf.set('fs.oci.client.auth.pemfilepath', "~/.oci/oci_api_key")
# spark.conf.set('fs.oci.client.auth.tenantId.region', "us-ashburn-1")
# spark.conf.set('fs.oci.client.hostname', "https://objectstorage.us-ashburn-1.oraclecloud.com")

spark.conf.set('fs.oci.client.auth.tenantId', "ocid1.tenancy.oc1..aaaaaaaamqxmfclvmrazpk4kt7ibkcfzfg2fvg4o2wyzemzu3n7tcpf6nvsa")
spark.conf.set('fs.oci.client.auth.userId', "ocid1.user.oc1..aaaaaaaa4gjumyz4kiowhzbyjzgajyo5bln565rntgboqod3t6lu4bpwl2ra")
spark.conf.set('fs.oci.client.auth.fingerprint', "ec:8f:0e:14:e0:31:cd:e4:d6:03:04:aa:aa:94:f9:a9")
spark.conf.set('fs.oci.client.auth.pemfilepath', "~/.oci/oci_api_key")
spark.conf.set('fs.oci.client.auth.tenantId.region', "us-ashburn-1")
spark.conf.set('fs.oci.client.hostname', "https://objectstorage.us-ashburn-1.oraclecloud.com")

# spark.driver.cores
# spark.driver.memory
# spark.driver.memoryOverhead
# spark.executor.memory = 1g
# spark.executor.memoryOverhead
# spark.driver.resource.{resourceName}.amount
# spark.executor.resource.{resourceName}.amount
# spark.executor.cores
spark.sparkContext._conf.getAll()  
#         .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
# .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

starting...
starting...


/opt/spark/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


[('spark.app.startTime', '1649669665325'),
 ('spark.driver.cores', '4'),
 ('spark.app.id', 'app-20220411093426-0011'),
 ('spark.driver.memory', '4g'),
 ('spark.scheduler.mode', 'FAIR'),
 ('spark.cores.max', '16'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir',
  'file:/home/centos/work/mobility_eval/spark-warehouse'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.executor.memory', '12g'),
 ('spark.executor.cores', '4'),
 ('spark.driver.host',
  'oke-cf2oghpidfa-nt5qvktoxqa-shjnqmwa77q-1.subc6544f31f.useastcluster.oraclevcn.com'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.master', 'spark://10.0.10.62:7077'),
 ('spark.driver.port', '40567'),
 ('spark.ui.showConsoleProgress', 'true')]

In [3]:
country = "IN"
def getPath(country):
    paths = {
        "US": "oci://ia-datapipeline@bmmp5bv7olp2/poi_data/USA_compressed_parquet_data/*.parquet",
#         "AR": "s3://ia-customer-insights/poi_data/ARE_processed_compressed_parquet_data/",
        "IN": "oci://ia-datapipeline@bmmp5bv7olp2/poi_data/all_IND_complete_compressed_parquet_data/*/*/*.parquet"
    }
    
    return paths[country]

block_path = getPath(country)

In [4]:

block_df = spark.read.format("parquet").load(block_path, header = 'true')

In [5]:
block_df.columns

['h3',
 'poiType',
 'poiCode',
 'h3index',
 'h12',
 'resolution',
 'index_res',
 'h6',
 '9_res',
 'state',
 'p_city']

In [6]:
read_path = "oci://ia-datapipeline@bmmp5bv7olp2/mobility_vendor_eval/onemata/IN/bronze_layers_all/date=2022-02-27"
df = spark.read.format('delta').load(read_path, header = 'true')

In [7]:
df.columns

['h3index9',
 'AAID',
 'latitude',
 'longitude',
 'timestamp',
 'device_type',
 'country',
 'hour',
 'day',
 'month',
 'h3index12',
 'state',
 'city',
 'time_precision_till_minutes',
 'h3index6']

In [ ]:
df.count()

In [8]:
df = df.withColumnRenamed("AAID", "ifa")
df = df.withColumnRenamed("h3index6", "h6")
df = df.withColumnRenamed("h3index12", "h12")
df = df.withColumnRenamed("h3index9", "h3")
df = df.withColumnRenamed("longitude", "long")
df = df.withColumnRenamed("latitude", "lat")
df = df.withColumnRenamed("latitude", "lat")

df.columns

['h3',
 'ifa',
 'lat',
 'long',
 'timestamp',
 'device_type',
 'country',
 'hour',
 'day',
 'month',
 'h12',
 'state',
 'city',
 'time_precision_till_minutes',
 'h6']

In [10]:
df = df.withColumn('date', F.to_date(F.col('timestamp')).cast(DateType()))

In [11]:
data_union = df.join(block_df.select(["h12","poiCode", "poiType"]), on=["h12"], how='inner').select(["h12", "h6", "h3", "ifa", "lat", "long", "date", "timestamp", "city","poiCode", "poiType"])
# data_union.show()

In [ ]:
data_union.write.partitionBy(['date', 'h6']).mode("overwrite").format('delta').save(vendor_mapping[vendor]["write_path"])